# Multispectral Band Metadata Update - v1.4.0 to v1.4.5

This notebook updates the metadata for multispectral bands from v1.4.0 to v1.4.5 configuration.

**Required Changes:**
- `*_9.TIF`: Red-650 (650nm) → Red edge-740 (740nm)
- `*_10.TIF`: Red edge-705 (705nm) → Red-650 (650nm)
- `*_11.TIF`: Red edge-740 (740nm) → Red edge-705 (705nm)


## 1. Setup and Dependencies


In [59]:
import os
import subprocess
import glob
from pathlib import Path

# Check if exiftool is installed
try:
    result = subprocess.run(['exiftool', '-ver'], capture_output=True, text=True)
    print(f"ExifTool version: {result.stdout.strip()}")
except FileNotFoundError:
    print("ExifTool not found. Please install it before proceeding.")
    print("Download from: https://exiftool.org/")


ExifTool version: 12.84


## 2. Create Config File

First, let's create the required config file for ExifTool.


In [60]:
def create_config_file(filename='jcpix4d.config'):
    """Create the ExifTool config file needed for metadata manipulation"""
    config_content = """%Image::ExifTool::UserDefined = (
    'Image::ExifTool::XMP::Main' => {
        Camera => { 
            SubDirectory => {
                TagTable => 'Image::ExifTool::UserDefined::Camera',
            },
        },
    },
); 

%Image::ExifTool::UserDefined::Camera = (
    GROUPS => { 0 => 'XMP', 1 => 'XMP-Camera', 2 => 'Camera' },
    NAMESPACE => { 'Camera' => 'http://pix4d.com/camera/1.0/'  },
    WRITABLE => 'string',
    Yaw             => { Writable => 'real' },
    Pitch           => { Writable => 'real' },
    Roll            => { Writable => 'real' },
    IMUSampleSize   => { Writable => 'integer' },
    IMUTimeOffset   => { Writable => 'integer' },
    LineReadoutTime => { Writable => 'integer' },
    IMUFrequency    => { Writable => 'real' },
    PrincipalPoint  => { },
    ModelType       => { },
    PerspectiveFocalLength => { Writable => 'real' },
    PerspectiveDistortion  => { },
    IMULinearVelocity => { },
    GPSXYAccuracy   => { Writable => 'real' },
    GPSZAccuracy    => { Writable => 'real' },
    FlightUUID      => { },
    CentralWavelength => { },
    CenterWavelength => { },
    WavelengthFWHM => { },
    Bandwidth => { },
    BandName        => { },
    RigName         => { },
    RigCameraIndex  => { },
    BandName        => { List => 'Seq' },
    IMUAngularVelocity => {
        Binary => 1,
        ValueConv => 'Image::ExifTool::XMP::DecodeBase64($val)',
        ValueConvInv => 'Image::ExifTool::XMP::EncodeBase64($val)',
    },
);
"""
    
    # Get current working directory for notebooks (since __file__ isn't available)
    root_dir = os.getcwd()
    config_path = os.path.join(root_dir, filename)
    
    # Write config file
    with open(config_path, 'w') as f:
        f.write(config_content)
    
    print(f"Created {filename} at: {config_path}")
    return config_path

# Create the config file
config_path = create_config_file('jcpix4d.config')


Created jcpix4d.config at: C:\Users\jcmontes\Documents\GitHub\dronescape_tools\_analysis\data-directory-notebooks\jcpix4d.config


## 3. Metadata Update Functions

Create specialized functions to update metadata for each band type.


In [61]:
def update_band_metadata(directory, band_suffix, band_name, wavelength, fwhm, bandwidth=None, dry_run=True, recursive=True):
    """Update metadata for a specific band type
    
    Args:
        directory (str): Directory containing the files
        band_suffix (str): Suffix identifying the band (e.g., '9' for *_9.TIF files)
        band_name (str): Band name to set
        wavelength (str): Wavelength value to set
        fwhm (str): Full Width Half Maximum value to set
        bandwidth (str, optional): Bandwidth value (if None, uses FWHM value)
        dry_run (bool): If True, only simulate the operation
        recursive (bool): If True, search for files recursively in subdirectories
        
    Returns:
        dict: Results of the operation
    """
    # Use FWHM as bandwidth if not specified
    if bandwidth is None:
        bandwidth = fwhm
    
    # Find matching files    
    if recursive:
        # Use recursive glob to find files in subdirectories
        pattern = f"**/*_{band_suffix}.TIF"
        files = list(Path(directory).glob(pattern))
        files = [str(f) for f in files]  # Convert Path objects to strings
    else:
        # Just search in the specified directory
        file_pattern = os.path.join(directory, f"*_{band_suffix}.TIF")
        files = glob.glob(file_pattern)
    
    if not files:
        return {'status': 'skipped', 'message': f"No files matching *_{band_suffix}.TIF found in {directory} or its subdirectories"}
    
    # Build command with path to config (in current working directory)
    root_dir = os.getcwd()
    config_path = os.path.join(root_dir, 'jcpix4d.config')
    
    # For recursive mode, use multiple commands for each file to ensure we only process expected files
    # For non-recursive mode, use wildcard pattern as before
    results = []
    if recursive:
        # Process each file individually 
        for file_path in files:
            cmd = [
                'exiftool',
                '-config', config_path,
                f'-XMP-Camera:BandName={band_name}',
                f'-XMP-Camera:CentralWavelength={wavelength}',
                f'-XMP-Camera:CenterWavelength={wavelength}',
                f'-XMP-Camera:WavelengthFWHM={fwhm}',
                f'-XMP-Camera:Bandwidth={bandwidth}',
                '-overwrite_original',  # Don't create backup files
            ]
            
            # Add dry run flag if specified
            if dry_run:
                cmd.append('-dryrun')
            
            # Add target file
            cmd.append(file_path)
            
            # Execute command
            print(f"Processing file: {os.path.basename(file_path)}")
            result = subprocess.run(cmd, capture_output=True, text=True)
            
            # Store individual result
            results.append({
                'file': file_path,
                'stdout': result.stdout,
                'stderr': result.stderr,
                'returncode': result.returncode,
            })
        
        # Combine results
        all_success = all(r['returncode'] == 0 for r in results)
        
        return {
            'files_matched': len(files),
            'files_processed': len(results),
            'individual_results': results,
            'returncode': 0 if all_success else 1,
            'status': 'success' if all_success else 'error'
        }
    else:
        # Non-recursive mode - use the original approach with wildcards
        file_pattern = os.path.join(directory, f"*_{band_suffix}.TIF")
        cmd = [
            'exiftool',
            '-config', config_path,
            f'-XMP-Camera:BandName={band_name}',
            f'-XMP-Camera:CentralWavelength={wavelength}',
            f'-XMP-Camera:CenterWavelength={wavelength}',
            f'-XMP-Camera:WavelengthFWHM={fwhm}',
            f'-XMP-Camera:Bandwidth={bandwidth}',
            '-overwrite_original',  # Don't create backup files
        ]
        
        # Add dry run flag if specified
        if dry_run:
            cmd.append('-dryrun')
        
        # Add target files
        cmd.append(file_pattern)
        
        # Execute command
        print(f"Running: {' '.join(cmd)}")
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        return {
            'files_matched': len(files),
            'stdout': result.stdout,
            'stderr': result.stderr,
            'returncode': result.returncode,
            'status': 'success' if result.returncode == 0 else 'error'
        }


## 4. Specialized Update Functions for v1.4.5

Create dedicated functions for each band update based on the v1.4.5 specifications.


In [62]:
def update_band_9_to_v145(directory, dry_run=True, recursive=True):
    """Update band 9 (*_9.TIF) from Red-650 to Red edge-740"""
    return update_band_metadata(
        directory=directory,
        band_suffix='9',
        band_name='Red edge-740',
        wavelength='740',
        fwhm='18',
        dry_run=dry_run,
        recursive=recursive
    )

def update_band_10_to_v145(directory, dry_run=True, recursive=True):
    """Update band 10 (*_10.TIF) from Red edge-705 to Red-650"""
    return update_band_metadata(
        directory=directory,
        band_suffix='10',
        band_name='Red-650',
        wavelength='650',
        fwhm='16',
        dry_run=dry_run,
        recursive=recursive
    )

def update_band_11_to_v145(directory, dry_run=True, recursive=True):
    """Update band 11 (*_11.TIF) from Red edge-740 to Red edge-705"""
    return update_band_metadata(
        directory=directory,
        band_suffix='11',
        band_name='Red edge-705',
        wavelength='705',
        fwhm='16',
        dry_run=dry_run,
        recursive=recursive
    )


## 5. Process Directory Function

A function to process all three bands in a single directory.


In [63]:
def process_directory_to_v145(directory, dry_run=True, recursive=True):
    """
    Process all three bands in a directory to update to v1.4.5
    
    Args:
        directory (str): Directory containing the files or parent directory with subdirectories
        dry_run (bool): If True, only simulate the operation
        recursive (bool): If True, search for files recursively in subdirectories
    """
    print(f"Processing directory: {directory}")
    if recursive:
        print("(Searching recursively in subdirectories)")
    
    results = {
        'directory': directory,
        'band_9': update_band_9_to_v145(directory, dry_run, recursive),
        'band_10': update_band_10_to_v145(directory, dry_run, recursive),
        'band_11': update_band_11_to_v145(directory, dry_run, recursive)
    }
    
    # Print summary
    print("\nResults summary:")
    for band, result in results.items():
        if band == 'directory':
            continue
        if result.get('status') == 'skipped':
            print(f"  {band}: Skipped - {result['message']}")
        else:
            status = "Success" if result.get('status') == 'success' else f"Error (code: {result.get('returncode')})"
            print(f"  {band}: {status}, {result.get('files_matched', 0)} files matched")
    
    return results


## 6. Batch Process Multiple Directories

Process a list of directories in batches.


In [64]:
def batch_process_directories(directories, batch_size=10, dry_run=True, recursive=True):
    """
    Process multiple directories in batches
    
    Args:
        directories (list): List of directories to process
        batch_size (int): Number of directories to process in each batch
        dry_run (bool): If True, only simulate the operation
        recursive (bool): If True, search for files recursively in subdirectories
    """
    total_dirs = len(directories)
    print(f"Processing {total_dirs} directories in batches of {batch_size}")
    
    # Ensure config file exists
    root_dir = os.getcwd()
    config_path = os.path.join(root_dir, 'jcpix4d.config')
    if not os.path.exists(config_path):
        create_config_file('jcpix4d.config')
    
    # Process in batches
    results = {}
    for i in range(0, total_dirs, batch_size):
        batch = directories[i:i+batch_size]
        batch_num = i//batch_size + 1
        total_batches = (total_dirs + batch_size - 1)//batch_size
        
        print(f"\n{'-'*50}")
        print(f"Batch {batch_num}/{total_batches} - Directories {i+1} to {min(i+batch_size, total_dirs)}")
        print(f"{'-'*50}")
        
        # Process each directory in this batch
        for dir_path in batch:
            dir_results = process_directory_to_v145(dir_path, dry_run, recursive)
            results[dir_path] = dir_results
            print("\n")
    
    # Final report
    print(f"\n{'='*50}")
    print(f"FINAL REPORT - {total_dirs} directories processed")
    print(f"{'='*50}")
    
    success_count = 0
    partial_count = 0
    failed_count = 0
    skipped_count = 0
    total_files_processed = 0
    
    for dir_path, result in results.items():
        band_statuses = [r.get('status') for k, r in result.items() if k != 'directory']
        
        # Check if all bands were skipped
        if all(s == 'skipped' for s in band_statuses):
            skipped_count += 1
            print(f"Skipped directory (no matching files): {dir_path}")
        elif all(s == 'success' for s in band_statuses):
            success_count += 1
            # Count files processed
            for k, r in result.items():
                if k != 'directory' and r.get('status') == 'success':
                    total_files_processed += r.get('files_matched', 0)
        elif all(s == 'error' for s in band_statuses):
            failed_count += 1
            print(f"Failed directory: {dir_path}")
        else:
            partial_count += 1
            # Count partially processed files
            for k, r in result.items():
                if k != 'directory' and r.get('status') == 'success':
                    total_files_processed += r.get('files_matched', 0)
    
    print(f"\nFully successful: {success_count}")
    print(f"Partially successful: {partial_count}")
    print(f"Failed: {failed_count}")
    print(f"Skipped (no matching files): {skipped_count}")
    print(f"Total files processed: {total_files_processed}")
    
    if dry_run:
        print("\nNOTE: This was a dry run. No files were actually modified.")
    
    return results


## 7. Test On A Single Directory

Let's test our functions on a single directory first.


In [19]:
# Set this to your test directory
# This should be a directory with the structure:
# <plotID>/YYYYMMDD/imagery/multispec/level0_raw/

# Example 1: Specify a full path to level0_raw directory 
test_directory = r"D:\TERN-Dronescape\NTABRT0001\20240829\imagery\multispec\level0_raw"

# Example 2: Use a higher level directory and search recursively for TIF files
# test_directory = r"D:\TERN-Dronescape\NTABRT0001"

# Run a dry test first with recursive search enabled
# print("Running dry test (no actual changes)...")
# test_results = process_directory_to_v145(test_directory, dry_run=False, recursive=True)


Processing directory: D:\TERN-Dronescape\NTABRT0001\20240829\imagery\multispec\level0_raw
(Searching recursively in subdirectories)
Processing file: IMG_0000_9.tif
Processing file: IMG_0001_9.tif
Processing file: IMG_0002_9.tif
Processing file: IMG_0003_9.tif
Processing file: IMG_0004_9.tif
Processing file: IMG_0005_9.tif
Processing file: IMG_0006_9.tif
Processing file: IMG_0007_9.tif
Processing file: IMG_0008_9.tif
Processing file: IMG_0009_9.tif
Processing file: IMG_0010_9.tif
Processing file: IMG_0011_9.tif
Processing file: IMG_0012_9.tif
Processing file: IMG_0013_9.tif
Processing file: IMG_0014_9.tif
Processing file: IMG_0015_9.tif
Processing file: IMG_0016_9.tif
Processing file: IMG_0017_9.tif
Processing file: IMG_0018_9.tif
Processing file: IMG_0019_9.tif
Processing file: IMG_0020_9.tif
Processing file: IMG_0021_9.tif
Processing file: IMG_0022_9.tif
Processing file: IMG_0023_9.tif
Processing file: IMG_0024_9.tif
Processing file: IMG_0025_9.tif
Processing file: IMG_0026_9.tif
Proc

## 8. Update Multiple Directories

Now you can provide a list of directories to process.
Fill in your directories in the list below.


In [93]:
# List of directories to update to v1.4.5
# You can specify either:
# 1. A list of level0_raw directories (more targeted, fewer subdirectories to search)
# 2. Higher-level directories with recursive search enabled (easier to specify fewer paths)

# Option 1: Targeted approach - specify level0_raw directories
directories_to_update = [
    # Add your level0_raw directories here, for example:
    r"D:\TERN-Dronescape\NTABRT0002\20240829\imagery\multispec\level0_raw",
    r"D:\TERN-Dronescape\NTABRT0003\20240829\imagery\multispec\level0_raw",
    r"D:\TERN-Dronescape\NTABRT0004\20240828\imagery\multispec\level0_raw"
]

# Option 2: High-level approach - use recursive search to find all subdirectories
# Set base_directory to your plot root and uncomment to find all matching directories
# base_directory = "D:\\TERN-Dronescape"
# directories_to_update = find_image_directories(base_directory, validate_structure=True)


In [ ]:
# First run a dry test with recursive search
print("Running dry test on all directories...")
# dry_results = batch_process_directories(directories_to_update, batch_size=5, dry_run=True, recursive=True)


## 9. Run Actual Update

If the dry run looks good, you can run the actual update.


In [94]:
# Ask for confirmation before running actual update
confirmation = input("Dry run complete. Do you want to proceed with actual metadata update? (yes/no): ")

if confirmation.lower() == 'yes':
    print("\nRunning actual metadata update (THIS WILL MODIFY FILES)...")
    real_results = batch_process_directories(directories_to_update, batch_size=5, dry_run=False, recursive=True)
    print("\nMetadata update complete!")
else:
    print("\nOperation cancelled. No files were modified.")


Dry run complete. Do you want to proceed with actual metadata update? (yes/no):  yes



Running actual metadata update (THIS WILL MODIFY FILES)...
Processing 3 directories in batches of 5

--------------------------------------------------
Batch 1/1 - Directories 1 to 3
--------------------------------------------------
Processing directory: D:\TERN-Dronescape\NTABRT0002\20240829\imagery\multispec\level0_raw
(Searching recursively in subdirectories)
Processing file: IMG_0000_9.tif
Processing file: IMG_0001_9.tif
Processing file: IMG_0002_9.tif
Processing file: IMG_0003_9.tif
Processing file: IMG_0004_9.tif
Processing file: IMG_0005_9.tif
Processing file: IMG_0006_9.tif
Processing file: IMG_0007_9.tif
Processing file: IMG_0008_9.tif
Processing file: IMG_0009_9.tif
Processing file: IMG_0010_9.tif
Processing file: IMG_0011_9.tif
Processing file: IMG_0012_9.tif
Processing file: IMG_0013_9.tif
Processing file: IMG_0014_9.tif
Processing file: IMG_0015_9.tif
Processing file: IMG_0016_9.tif
Processing file: IMG_0017_9.tif
Processing file: IMG_0018_9.tif
Processing file: IMG_0019

## 10. Advanced: Search for Image Directories

This section helps find directories that contain the TIF files.


In [ ]:
def find_image_directories(base_dir, suffixes=['9', '10', '11'], validate_structure=True):
    """
    Find directories containing TIF files with the specified suffixes
    
    Args:
        base_dir (str): Base directory to search from
        suffixes (list): List of band suffixes to look for
        validate_structure (bool): If True, only include directories that match the 
                                  <plotID>/YYYYMMDD/imagery/multispec/level0_raw/ pattern
                                  
    Returns:
        list: Sorted list of directories containing matching TIF files
    """
    image_dirs = set()
    
    print(f"Searching for image directories in {base_dir}...")
    
    # Walk through the directory tree
    for root, dirs, files in os.walk(base_dir):
        has_matching_files = False
        
        # Check if any files match our pattern
        for suffix in suffixes:
            pattern = f"*_{suffix}.TIF"
            matching_files = glob.glob(os.path.join(root, pattern))
            if matching_files:
                has_matching_files = True
                break
        
        if has_matching_files:
            # Check if directory structure matches expected pattern if validation is enabled
            if validate_structure:
                # Convert to normalized Path object
                path = Path(root).resolve()
                
                # Check if this directory follows the pattern
                # <plotID>/YYYYMMDD/imagery/multispec/level0_raw/...
                parts = path.parts
                
                # Look for 'level0_raw' and check the preceding parts
                if 'level0_raw' in parts:
                    level0_index = parts.index('level0_raw')
                    
                    # Need at least 4 parts before level0_raw for the pattern to be valid
                    if level0_index >= 4:
                        # Check that preceding parts match pattern
                        if parts[level0_index-1] == 'multispec' and parts[level0_index-2] == 'imagery':
                            # Date part should be YYYYMMDD (8 digits)
                            date_part = parts[level0_index-3]
                            if len(date_part) == 8 and date_part.isdigit():
                                # Plot ID should be non-empty
                                if parts[level0_index-4]:
                                    image_dirs.add(root)
                            else:
                                # If date doesn't match pattern but we still want the directory
                                # Uncomment the line below to include directories with invalid dates
                                # image_dirs.add(root)
                                pass
            else:
                # If validation is disabled, include all directories with matching files
                image_dirs.add(root)
    
    # Convert to sorted list
    result = sorted(list(image_dirs))
    print(f"Found {len(result)} directories with matching TIF files")
    
    return result


In [ ]:
# Example: Find all image directories within a base directory that match the pattern
base_search_directory = "D:\\TERN-Dronescape\\NTABRT0001"  # Replace with your actual path
# found_directories = find_image_directories(base_search_directory, validate_structure=True)

# Print the directories found (uncomment to use)
# print("Directories that match the pattern <plotID>/YYYYMMDD/imagery/multispec/level0_raw/:")
# for i, directory in enumerate(found_directories):
#     print(f"{i+1}. {directory}")

# You can then use these directories for batch processing:
# directories_to_update = found_directories

# If you want to find ALL directories with matching TIF files regardless of structure:
# all_dirs = find_image_directories(base_search_directory, validate_structure=False)


## 8. Update Multiple Directories

Now you can provide a list of directories to process.
Fill in your directories in the list below.
